In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
from explorer import Explorer

In [ ]:
e = Explorer()

In [ ]:
for i in range(30):
    e.explore()
    e.refine_experience()